In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import read_csv
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lazypredict.Supervised import LazyClassifier, LazyRegressor

In [ ]:
df = read_csv('Social_Network_Ads.csv')
display(df)
print(df.info())

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB
None


In [ ]:
# profile = ProfileReport(df, title = "Social Network Ads", explorative = True)
# profile.to_file("Social_Network_Ads_Report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df = df.drop('User ID', axis = 1)
target = 'Purchased'
x = df.drop(target, axis = 1)
y = df[target]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
df[['Age', 'EstimatedSalary']].plot(kind='box', subplots=True, layout=(1, 2), figsize=(10, 4), sharex=False)
plt.savefig('box_test.png')
plt.show()

In [ ]:
num_trans = Pipeline(steps = [
    ('scaler', StandardScaler())
])
nom_trans = Pipeline(steps = [
    ('encoder', OneHotEncoder(sparse_output = False))
])

preprocess = ColumnTransformer(transformers = [
    ('num_trans', num_trans, ['Age', 'EstimatedSalary']),
    ('nom_trans', nom_trans, ['Gender'])
])

In [63]:
x_train = preprocess.fit_transform(x_train)
x_test = preprocess.transform(x_test)

In [151]:
model = MLPClassifier(hidden_layer_sizes = 50)
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95        52
           1       0.90      0.93      0.91        28

    accuracy                           0.94        80
   macro avg       0.93      0.94      0.93        80
weighted avg       0.94      0.94      0.94        80



In [115]:
clf = LazyClassifier(verbose = 0, ignore_warnings = False, custom_metric = None, random_state = 42)
models, prediction = clf.fit(x_train, x_test, y_train, y_test)
print(models)

 47%|████▋     | 15/32 [00:00<00:00, 75.71it/s]

CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X).
FixedThresholdClassifier model failed to execute
FixedThresholdClassifier.__init__() missing 1 required positional argument: 'estimator'


100%|██████████| 32/32 [00:00<00:00, 39.89it/s]

SelfTrainingClassifier model failed to execute
You must pass an estimator to SelfTrainingClassifier. Use `estimator`.
StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
TunedThresholdClassifierCV model failed to execute
TunedThresholdClassifierCV.__init__() missing 1 required positional argument: 'estimator'
[LightGBM] [Info] Number of positive: 115, number of negative: 205
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000120 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 119
[LightGBM] [Info] Number of data points in the train set: 320, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.359375 -> initscore=-0.578078
[LightGBM] [Info] Start training from score -0.578078
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

GRIDSEARCH FOR LOGISTIC REGRESSION

In [ ]:
model = LogisticRegression(random_state = 42, n_jobs = -1)

LogisticRegression_Param = {
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    'penalty': ['l1', 'l2', 'elasticnet', None],
    'tol': [1e-3, 1e-4, 1e-5],
    'fit_intercept': [True, False],
    'class_weight': ['balanced', None],
    'max_iter': [50, 100, 150]
}


LogisticRegression_Grid = GridSearchCV(estimator = model, param_grid = LogisticRegression_Param, scoring = "accuracy", verbose = 2, n_jobs = -1)
LogisticRegression_Grid.fit(x_train, y_train)
y_predict = LogisticRegression_Grid.predict(x_test)
print(LogisticRegression_Grid.best_params_)
print(LogisticRegression_Grid.best_score_)
print(classification_report(y_test, y_predict))

Fitting 5 folds for each of 864 candidates, totalling 4320 fits
{'class_weight': 'balanced', 'fit_intercept': True, 'max_iter': 50, 'penalty': 'l1', 'solver': 'saga', 'tol': 0.001}
0.8375
              precision    recall  f1-score   support

           0       0.92      0.90      0.91        52
           1       0.83      0.86      0.84        28

    accuracy                           0.89        80
   macro avg       0.87      0.88      0.88        80
weighted avg       0.89      0.89      0.89        80



GRIDSEARCH FOR RANDOMFOREST CLASSIFIER

In [ ]:
model = RandomForestClassifier(random_state = 42, n_jobs = -1)

# RandomForestClassifier_Param = {
#     'n_estimators': [10, 30, 50, 70, 100, 120, 150],
#     'criterion': ["gini", "log_loss", "entropy"], 
#     'max_depth': [None, 3, 5, 7, 10],
#     'min_samples_split': [2, 3, 5, 10],
#     'min_samples_leaf': [1, 2, 3],
#     'max_features': [None, "log2", "sqrt"],
#     'class_weight': [None, "balanced"]
# }
RandomForestClassifier_Param = {
    'n_estimators': [30, 50, 70, 100, 120],
    'criterion': ["gini", "log_loss", "entropy"], 
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3],
}


RandomForestClassifier_Grid = GridSearchCV(estimator = model, param_grid = RandomForestClassifier_Param, scoring = "accuracy", verbose = 2, n_jobs = -1)
RandomForestClassifier_Grid.fit(x_train, y_train)
y_predict = RandomForestClassifier_Grid.predict(x_test)
print(RandomForestClassifier_Grid.best_params_)
print(RandomForestClassifier_Grid.best_score_)
print(classification_report(y_test, y_predict))

Fitting 5 folds for each of 540 candidates, totalling 2700 fits
{'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 30}
0.909375
              precision    recall  f1-score   support

           0       0.96      0.90      0.93        52
           1       0.84      0.93      0.88        28

    accuracy                           0.91        80
   macro avg       0.90      0.92      0.91        80
weighted avg       0.92      0.91      0.91        80



GRIDSEARCH FOR SVC

In [ ]:
model = svm.SVC(random_state = 42)

common_params = {
    'tol': [1e-3, 1e-4],
    'C': [0.1, 1, 10]
}

branch_specific = [
    {'kernel': ['linear']},
    {'kernel': ['poly'], 'degree': [2, 3], 'coef0': [0.0, 0.5], 'gamma': ['scale', 'auto']},
    {'kernel': ['rbf'], 'gamma': ['scale', 'auto']},
    {'kernel': ['sigmoid'], 'gamma': ['scale'], 'coef0': [0.0, 0.5]}
]

SVC_Param = []
for branch in branch_specific:
    merged = branch.copy()
    merged.update(common_params)
    SVC_Param.append(merged)


SVM_Grid = GridSearchCV(estimator = model, param_grid = SVC_Param, scoring = "accuracy", verbose = 2, n_jobs = -1)
SVM_Grid.fit(x_train, y_train)
y_predict = SVM_Grid.predict(x_test)
print(SVM_Grid.best_params_)
print(SVM_Grid.best_score_)
print(classification_report(y_test, y_predict))

Fitting 5 folds for each of 78 candidates, totalling 390 fits
{'C': 10, 'gamma': 'scale', 'kernel': 'rbf', 'tol': 0.001}
0.9125
              precision    recall  f1-score   support

           0       0.96      0.92      0.94        52
           1       0.87      0.93      0.90        28

    accuracy                           0.93        80
   macro avg       0.91      0.93      0.92        80
weighted avg       0.93      0.93      0.93        80



GRIDSEARCH FOR MLPCLASSIFIER

In [156]:
model = MLPClassifier(random_state = 42)

MLP_Param = {
   'hidden_layer_sizes': [50, 70, 100, 120, 150],
   'activation': ['tanh', 'relu'],
   'solver': ['lbfgs', 'sgd', 'adam'],
   'tol': [1e-3, 1e-4],
}


MLP_Grid = GridSearchCV(estimator = model, param_grid = MLP_Param, scoring = 'accuracy', verbose = 2, n_jobs = -1)
MLP_Grid.fit(x_train, y_train)
y_predict = MLP_Grid.predict(x_test)
print(MLP_Grid.best_params_)
print(MLP_Grid.best_score_)
print(classification_report(y_test, y_predict))

Fitting 5 folds for each of 60 candidates, totalling 300 fits
{'activation': 'relu', 'hidden_layer_sizes': 100, 'solver': 'adam', 'tol': 0.0001}
0.903125
              precision    recall  f1-score   support

           0       0.96      0.94      0.95        52
           1       0.90      0.93      0.91        28

    accuracy                           0.94        80
   macro avg       0.93      0.94      0.93        80
weighted avg       0.94      0.94      0.94        80



REMOVE "AGE" FEATURE

In [ ]:
x_train_rm_age = x_train[:, [1, 2, 3]]
x_test_rm_age = x_test[:, [1, 2, 3]]
y_train_rm_age = y_train
y_test_rm_age = y_test

In [185]:
model = MLPClassifier(random_state = 42)
model.fit(x_train_rm_age, y_train_rm_age)
y_predict = model.predict(x_test_rm_age)
print(classification_report(y_test_rm_age, y_predict))

              precision    recall  f1-score   support

           0       0.73      0.94      0.82        52
           1       0.77      0.36      0.49        28

    accuracy                           0.74        80
   macro avg       0.75      0.65      0.66        80
weighted avg       0.74      0.74      0.71        80



In [180]:
df1 = df.copy()
target = 'Purchased'
x1 = df.drop([target, "Age"], axis = 1)
y1 = df[target]
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size = 0.2, random_state = 42)
display(x_train1)

test = ColumnTransformer(transformers=[
    ('num', StandardScaler(), ['EstimatedSalary']),
    ('cat', OneHotEncoder(sparse_output=False), ['Gender'])
])

x_train1 = test.fit_transform(x_train1)
x_test1 = test.transform(x_test1)
display(x_train1)
display(x_test1)

,Gender,EstimatedSalary
3,Female,57000
18,Male,28000
202,Female,134000
250,Female,39000
274,Female,26000
...,...,...
71,Female,27000
106,Female,35000
270,Female,133000
348,Male,77000


array([[-0.38634438,  1.        ,  0.        ],
       [-1.22993871,  0.        ,  1.        ],
       [ 1.853544  ,  1.        ,  0.        ],
       [-0.90995465,  1.        ,  0.        ],
       [-1.28811763,  1.        ,  0.        ],
       [ 1.44629156,  0.        ,  1.        ],
       [-0.53179168,  0.        ,  1.        ],
       [ 0.10817643,  0.        ,  1.        ],
       [ 0.45724994,  0.        ,  1.        ],
       [ 0.31180264,  0.        ,  1.        ],
       [ 0.45724994,  0.        ,  1.        ],
       [-0.47361276,  1.        ,  0.        ],
       [-0.32816546,  0.        ,  1.        ],
       [ 0.28271318,  1.        ,  0.        ],
       [-1.57901222,  0.        ,  1.        ],
       [ 0.25362372,  1.        ,  0.        ],
       [ 1.73718616,  0.        ,  1.        ],
       [-0.18271817,  0.        ,  1.        ],
       [ 2.11534913,  1.        ,  0.        ],
       [ 1.21357589,  0.        ,  1.        ],
       [ 1.50447048,  1.        ,  0.   

array([[-1.40447546,  1.        ,  0.        ],
       [ 0.51542886,  1.        ,  0.        ],
       [-0.76450736,  1.        ,  0.        ],
       [ 0.74814454,  1.        ,  0.        ],
       [-1.22993871,  1.        ,  0.        ],
       [-0.24089709,  1.        ,  0.        ],
       [ 1.06812859,  1.        ,  0.        ],
       [ 0.36998156,  1.        ,  0.        ],
       [ 0.13726589,  0.        ,  1.        ],
       [-0.15362871,  0.        ,  1.        ],
       [-0.15362871,  1.        ,  0.        ],
       [-1.05540195,  1.        ,  0.        ],
       [-0.64814952,  1.        ,  0.        ],
       [-1.37538601,  1.        ,  0.        ],
       [ 0.4863394 ,  0.        ,  1.        ],
       [ 1.09721805,  0.        ,  1.        ],
       [-0.93904411,  1.        ,  0.        ],
       [ 0.10817643,  1.        ,  0.        ],
       [-0.82268628,  0.        ,  1.        ],
       [-0.29907601,  0.        ,  1.        ],
       [-1.25902817,  1.        ,  0.   

REMOVE ESTIMATED SALARY

In [188]:
x_train_rm_salary = x_train[:, [0, 2, 3]]
x_test_rm_salary = x_test[:, [0, 2, 3]]
y_train_rm_salary = y_train
y_test_rm_salary = y_test

In [193]:
model = LogisticRegression(random_state = 42)
model.fit(x_train_rm_salary, y_train_rm_salary)
y_predict = model.predict(x_test_rm_salary)
print(classification_report(y_test_rm_salary, y_predict))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93        52
           1       0.92      0.82      0.87        28

    accuracy                           0.91        80
   macro avg       0.91      0.89      0.90        80
weighted avg       0.91      0.91      0.91        80



REMOVE GENDER

In [195]:
x_train_rm_gender = x_train[:, [0, 1]]
x_test_rm_gender = x_test[:, [0, 1]]
y_train_rm_gender = y_train
y_test_rm_gender = y_test

In [199]:
model = MLPClassifier(random_state = 42)
model.fit(x_train_rm_gender, y_train_rm_gender)
y_predict = model.predict(x_test_rm_gender)
print(classification_report(y_test_rm_gender, y_predict))

              precision    recall  f1-score   support

           0       0.98      0.92      0.95        52
           1       0.87      0.96      0.92        28

    accuracy                           0.94        80
   macro avg       0.93      0.94      0.93        80
weighted avg       0.94      0.94      0.94        80

